<a href="https://colab.research.google.com/github/deblinabarua/RAG_basic/blob/main/Implementing_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain_community

In [ ]:
pip install langchain_chroma

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [ ]:
pip install unstructured

In [ ]:
pip install -U langchain-huggingface

In [ ]:
!pip install numpy==1.24.3 scipy==1.10.1

  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.21 requires numpy<3,>=1.26.2, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires num

In [ ]:
!python --version

Python 3.11.11


In [ ]:
import transformers.generation.utils

In [ ]:
import transformers.integrations.integration_utils

In [ ]:
!pip uninstall -y jax jaxlib transformers
!pip install --upgrade transformers
!pip install --upgrade "jax[cpu]"  # or jax[cuda] if using GPU

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
Found existing installation: transformers 4.51.1
Uninstalling transformers-4.51.1:
  Successfully uninstalled transformers-4.51.1
  Using cached transformers-4.51.1-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.1-py3-none-any.whl (10.4 MB)


   ━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━ 64.0/105.1 MB 65.8 MB/s eta 0:00:01
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install -U numpy scipy
!pip install -U transformers datasets tokenizers
!pip install -U langchain-huggingface

In [ ]:
pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 8.9 MB/s eta 0:00:00


In [ ]:
pip install huggingface_hub[hf_xet]

In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document
from IPython.display import Markdown, display
import ipywidgets as widgets
import shutil
from langchain_chroma import Chroma
import numpy
from langchain_huggingface import HuggingFaceEmbeddings

# Configuration
CHROMA_PATH = "/content/chroma"
DATA_PATH = "/content/rag"  # Path to your markdown files
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
SIMILARITY_THRESHOLD = 0.7
TOP_K_RESULTS = 3

PROMPT_TEMPLATE = """
Answer the question based only on the following medical oncology context:

{context}

---

Question: {question}

Provide a concise, accurate answer using only the provided medical context.
If you don't know the answer, say you don't know.
"""

def load_documents():
    """Load markdown documents from the specified directory."""
    try:
        loader = DirectoryLoader(DATA_PATH, glob="*.md")
        documents = loader.load()
        print(f"Loaded {len(documents)} documents")
        return documents
    except Exception as e:
        raise RuntimeError(f"Failed to load documents: {str(e)}")

def split_documents(documents: list[Document]):
    """Split documents into chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks")
    return chunks

def create_vector_store(chunks: list[Document]):
    """Create and persist Chroma vector store."""
    if os.path.exists(CHROMA_PATH):
        print("Clearing existing Chroma database")

        shutil.rmtree(CHROMA_PATH)

    print(f"Creating embeddings with {MODEL_NAME}")
    embeddings = HuggingFaceEmbeddings(
        model_name=MODEL_NAME,
        model_kwargs={'device': 'cpu'},  # Change to 'cuda' if GPU available
        encode_kwargs={'normalize_embeddings': False}
    )

    db = Chroma.from_documents(
        documents = chunks,
        embedding = embeddings,
        persist_directory=CHROMA_PATH
    )
    #db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}")
    return db

def initialize_vector_store():
    """Initialize or load the vector store."""
    if os.path.exists(CHROMA_PATH):
        print("Loading existing Chroma database")
        embeddings = HuggingFaceEmbeddings(model_name=MODEL_NAME)
        return Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    else:
        print("Creating new Chroma database")
        documents = load_documents()
        chunks = split_documents(documents)
        return create_vector_store(chunks)

def query_database(db, query_text):
    """Query the database for relevant documents."""
    try:
        results = db.similarity_search_with_relevance_scores(
            query_text,
            k=TOP_K_RESULTS
        )
        if not results or results[0][1] < SIMILARITY_THRESHOLD:
            return None, None

        context_text = "\n\n---\n\n".join([doc.page_content for doc, _ in results])
        sources = [doc.metadata.get("source", "Unknown") for doc, _ in results]
        return context_text, sources
    except Exception as e:
        raise RuntimeError(f"Database query failed: {str(e)}")

def generate_response(prompt):
    """Generate response using the prompt template."""
    try:
        # For medical Q&A, we'll use the prompt directly since we're not using an LLM here
        # In a real implementation, you would use an LLM like Flan-T5 or Med-PaLM
        return prompt
    except Exception as e:
        raise RuntimeError(f"Response generation failed: {str(e)}")

def rag_query(query_text):
    """Complete RAG pipeline for a query."""
    try:
        # Load vector store
        display(Markdown(f"🔍 *Loading medical knowledge base...*"))
        db = initialize_vector_store()

        # Search for relevant documents
        display(Markdown(f"🔎 *Searching for relevant medical information...*"))
        context_text, sources = query_database(db, query_text)

        if not context_text:
            display(Markdown("❌ *No sufficiently relevant medical information found.*"))
            return

        # Format prompt
        prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
        prompt = prompt_template.format(context=context_text, question=query_text)

        # Generate response (in a real implementation, this would use an LLM)
        display(Markdown("🤖 *Generating medical response...*"))
        response_text = generate_response(prompt)

        # Display results
        display(Markdown(f"## 📢 Medical Response\n{response_text}"))
        display(Markdown("## 📚 Source Documents"))
        for source in sources:
            display(Markdown(f"- {source}"))

    except Exception as e:
        display(Markdown(f"❌ *An error occurred:* {str(e)}"))

# Create Colab interface
query_input = widgets.Text(
    placeholder='Enter your medical oncology question...',
    description='Question:',
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(description="Ask")
output = widgets.Output()

def on_click(b):
    with output:
        output.clear_output()
        if query_input.value.strip():
            rag_query(query_input.value)
        else:
            display(Markdown("⚠️ Please enter a question"))

submit_button.on_click(on_click)

# Display the interface
display(widgets.VBox([
    widgets.HTML("<h1>Medical Oncology Handbook Q&A</h1>"),
    widgets.HTML("<p>Ask questions about the Medical Oncology Handbook for Junior Medical Officers</p>"),
    query_input,
    submit_button,
    output
]))

# Example usage
# rag_query("What are the common side effects of 5-FU?")
# rag_query("How is febrile neutropenia managed?")